In [1]:
import pandas as pd
import numpy as np
import wfdb
from wfdb import processing
import ast
import scipy as sp
from scipy import signal
from scipy import stats
from hampel import hampel
from dtaidistance.subsequence.dtw import subsequence_alignment
from dtaidistance import dtw_visualisation as dtwvis
import matplotlib.pyplot as plt
from itertools import groupby
from operator import itemgetter
from scipy.signal import savgol_filter
import heartpy as hp
from scipy.signal import find_peaks
from findpeaks import findpeaks
import neurokit2 as nk
from scipy.signal import butter, lfilter
from scipy.signal import freqz
import seaborn as sns
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing







In [2]:
DATA_PATH = './data'
path = DATA_PATH + '/ptbxl_database.csv'
df = pd.read_csv(path)
path_stats = DATA_PATH + '/scp_statements.csv'
df_statements = pd.read_csv(path_stats) 
df_statements.rename(columns={'Unnamed: 0': 'Abreviated'}, inplace=True)

In [3]:

current_idx = df_statements.index[df_statements['Abreviated'] == 'ISCAL']

In [4]:
print(df.head())
print(df["filename_hr"][0])

   ecg_id  patient_id   age  sex  height  weight  nurse  site     device  \
0       1     15709.0  56.0    1     NaN    63.0    2.0   0.0  CS-12   E   
1       2     13243.0  19.0    0     NaN    70.0    2.0   0.0  CS-12   E   
2       3     20372.0  37.0    1     NaN    69.0    2.0   0.0  CS-12   E   
3       4     17014.0  24.0    0     NaN    82.0    2.0   0.0  CS-12   E   
4       5     17448.0  19.0    1     NaN    70.0    2.0   0.0  CS-12   E   

        recording_date  ... SLI-LVH QRS-CLBB ST-ELEV-MI ST-DEPR-MI  Q-ISC  \
0  1984-11-09 09:17:34  ...   False    False      False      False  False   
1  1984-11-14 12:55:37  ...    True    False      False      False  False   
2  1984-11-15 12:49:10  ...   False    False      False      False  False   
3  1984-11-15 13:44:57  ...   False    False      False      False  False   
4  1984-11-17 10:43:15  ...   False    False      False      False  False   

   Q-ISC-QPeak  Q-ISC-V2V3  Q-ISC-RPeak  STRAIN MI-ALL  
0        False       Fa

In [49]:
def extract_features(df=None):
    P_mean_length = []
    P_mean_amplitude = []
    Q_mean_length = []
    Q_mean_amplitude = []
    R_mean_length = []
    R_mean_amplitude = []
    S_mean_length = []
    S_mean_amplitude = []
    T_mean_length = []
    T_mean_amplitude = []
    P_power = []
    Q_power = []
    R_power = []
    S_power = []
    T_power = []
    P_std_powers = []
    Q_std_powers = []
    R_std_powers = []
    T_std_powers = []
    S_std_powers = []
    for n in range(len(df)):
    # for n in range(len(df)):
        data = wfdb.rdrecord('/Users/alexenabernad/Desktop/Idoven test/Idoven-Data-Scientist/physionet.org/files/ptb-xl/1.0.2/'+ df["filename_hr"][n])
    # wfdb.plot_wfdb(record=data, title='10 s ECG') 
        display(data.__dict__)
        
        fs=500

        #The R peaks indexes refer to the 100 Hz signal, for the 500Hz signal the peaks are multiplied by 5
        # so the R 
        # plt.plot(signal.detrend(data.p_signal[:,0]))

        lowcut = 5
        highcut=20



        def butter_bandpass(lowcut, highcut, fs, order=5):
            return butter(order, [lowcut, highcut], fs=fs, btype='bandpass')

        def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
            b, a = butter_bandpass(lowcut, highcut, fs, order=order)
            y = lfilter(b, a, data)
            return y
        sos = signal.butter(6,0.5,fs=fs,btype='highpass',output='sos')
        sos = signal.butter(6,1,fs=500,btype='highpass',output='sos')
        b=signal.sosfilt(sos,signal.detrend(data.p_signal[:,0]))
        signal_highpassed = b
        y = butter_bandpass_filter(b, lowcut, highcut, fs, order=6)
        savgol_ecg = savgol_filter(y,window_length = 25, polyorder=2)


        #Get a signal product of the sum of the 12 channels
        def get_total_signal():
            channels = data.p_signal[:,0]
            for i in range(12):
                channels = channels+data.p_signal[:,i]
                
            # plt.plot(channels)
            baseline = hp.remove_baseline_wander(channels,sample_rate=fs,cutoff = 0.002)
            return baseline
        baseline = get_total_signal()
        ##Method to find P,R,S,T peaks using Hampel Filter

        def peaks_hampel():
            # fp = findpeaks(method= 'peakdetect',lookahead=40,interpolate=None)
            # results = fp.fit(baseline[:1500])['df']
            # peaks_valleys = results.loc[results['valley']|results["peak"],"x"].values
            baseline_zero_centered = baseline-np.mean(baseline)
            outlier_indices = hampel(pd.Series(baseline_zero_centered), window_size=100, n=3)

            peaks = [list(map(itemgetter(1), g)) for k, g in groupby(enumerate(outlier_indices), lambda x: x[0]-x[1])]
            peaks_max_vals = [peaks[i][np.argmax(abs(baseline_zero_centered[peaks[i]]))] for i in range(len(peaks))]
            peaks_sign = np.sign(baseline_zero_centered[peaks_max_vals])
            diffs_max = np.where((np.diff(peaks_max_vals)<40))[0]
            zipped_list = list(zip(baseline_zero_centered[peaks_max_vals],peaks_sign,peaks_max_vals))

            counter = 0

            for i in diffs_max:
                a = i + 1
                if zipped_list[i][0] > zipped_list[a][0] and zipped_list[i][1]==zipped_list[a][1]:
                    peaks_max_vals.pop(a-counter)
                    counter += 1
                if zipped_list[i][0] < zipped_list[a][0] and zipped_list[i][1]==zipped_list[a][1]:
                    peaks_max_vals.pop(i-counter)
                    counter +=1
                else:
                    pass

            # plt.plot(peaks_max_vals,baseline_zero_centered[peaks_max_vals],'x')
            # plt.plot(baseline_zero_centered)
            # print(peaks_max_vals)
            return baseline_zero_centered,peaks_max_vals

        baseline_zero_centered,peaks_max_vals = peaks_hampel()
        #With this method all the P,R,S,T peaks can be detected. The problem surges in how to separate the R peaks and how to find the Q peaks. 
        #R peaks can be detected due to the fact that those peaks have higher amplitude than the surrounding peaks, so a window  in the peaks list that takes the peaks 4 by 4 and extract the biggest value 
        #--> Can check which is the R peak
        #For the Q wave detection, the solution is simpler once the R is detected. It corresponds to the min value between R and P waves.
        #There are libraries in internet that automatically calculate the peaks like the neurokit2. https://neuropsychology.github.io/NeuroKit/examples/ecg_delineate/ecg_delineate.html
        #This library is very useful but needs the method developed in this point to be more accurate. Neurokit is imported as nk..
        _, rpeaks = nk.ecg_peaks(baseline_zero_centered, sampling_rate=500)
        #This method works very well but has as problem that it does not detect the first r-peak properly.
        #Here acts the local points of interest extracted in the last section. Now all the R peaks are extracted
        first_peaks = np.where(peaks_max_vals<=rpeaks["ECG_R_Peaks"][0])
        first_peaks = [peaks_max_vals[first_peaks[0][i]] for i in first_peaks[0]]
        if len(first_peaks)>0:
            first_r_peak = first_peaks[np.argmax(baseline_zero_centered[first_peaks])]
        if first_r_peak < rpeaks["ECG_R_Peaks"][0]:
            rpeaks["ECG_R_Peaks"] = np.insert(rpeaks["ECG_R_Peaks"],0,first_r_peak,axis=0)
        _, waves_peak = nk.ecg_delineate(baseline_zero_centered, rpeaks, sampling_rate=500, method="peak")
        # plot = nk.events_plot([rpeaks["ECG_R_Peaks"][:1],
        #                     waves_peak['ECG_T_Peaks'][:1], 
        #                     waves_peak['ECG_P_Peaks'][:1],
        #                     waves_peak['ECG_Q_Peaks'][:1],
        #                     waves_peak['ECG_S_Peaks'][:1]], baseline_zero_centered[:500])
        
        #Can be plotted as utility
        
        # signal_dwt, waves_dwt = nk.ecg_delineate(baseline_zero_centered, 
        #                                         rpeaks, 
        #                                         sampling_rate=500, 
        #                                         method="dwt", 
        #                                         show=True, 
        #                                         show_type='peaks')
        
        #Can be plotted in order to understand the signal shape better


        #Once calculated all the peaks, some metrics can be extracted:
        #The len of the data is 5000, with a frequency sample rate of 500 Hz it means that the total time is 10seconds. The heartrate can be calculated using that information and the  R peaks
        heartrate = len(rpeaks["ECG_R_Peaks"])*6
        heartbeats_tot = len(rpeaks["ECG_R_Peaks"])

        # print(waves_peak['ECG_P_Peaks'])
        # print(waves_peak['ECG_Q_Peaks'])
        # print(rpeaks["ECG_R_Peaks"])
        # print(waves_peak['ECG_S_Peaks'])
        # print(waves_peak['ECG_T_Peaks'])
        def extract_ends(signal = None,point = None):
            if point!= None:
                sign = np.sign(signal[point])
            zero_left = np.where(sign==np.sign(signal[:point]))
            zero_left = [list(map(itemgetter(1), g)) for k, g in groupby(enumerate(zero_left[0]), lambda x: x[0]-x[1])]
            # print(zero_left[-1][0])
            if len(zero_left)>0: 
                zero_left = zero_left[-1][0]
            else:
                zero_left = 0 ##Provisional solution
            
            zero_right = np.where(sign==np.sign(signal[point:]))
            zero_right = [list(map(itemgetter(1), g)) for k, g in groupby(enumerate(zero_right[0]), lambda x: x[0]-x[1])]
            # print(zero_right[0][-1])
            if len(zero_right)>0:
                zero_right = zero_right[0][-1] + point
            else: 
                zero_right = len(signal)
                
            if zero_right > len(signal):
                zero_right = point
            if len(np.where(sign==np.sign(signal[:point]))[0]) == 0:
                zero_len = 0 
            return zero_left,zero_right
        
        def wave_length(signal=None,point=None):
            if np.isnan(point) == True:
                length = 0
            else:
                zero_left,zero_right = extract_ends(signal=signal,point=point)
                length = (zero_right-zero_left)/500
            return(length)
        def wave_powers(signal=None,point=None):
            if np.isnan(point) == True:
                tot_energy = 0
            else:
                zero_left,zero_right = extract_ends(signal=signal,point=point)
                rms = np.sqrt(np.sum(signal[zero_left:zero_right]**2)/(zero_right-zero_left))
                tot_energy = rms**2
            return tot_energy
        #Extraction of lengths and amplitudes of the different waves(width and heigh)
        P_wave_lengths = [wave_length(signal=baseline_zero_centered,point=waves_peak["ECG_P_Peaks"][i]) for i in range(len(waves_peak["ECG_P_Peaks"]))]
        P_amplitudes = [baseline_zero_centered[waves_peak["ECG_P_Peaks"][i]] if np.isnan(waves_peak["ECG_P_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_P_Peaks"]))]
        P_mean_length.append(np.mean(P_wave_lengths))
        P_mean_amplitude.append(np.mean(P_amplitudes))
        Q_wave_lengths= [wave_length(signal=baseline_zero_centered,point=waves_peak["ECG_Q_Peaks"][i]) for i in range(len(waves_peak["ECG_Q_Peaks"]))]
        Q_amplitudes = [baseline_zero_centered[waves_peak["ECG_Q_Peaks"][i]] if np.isnan(waves_peak["ECG_Q_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_Q_Peaks"]))]
        Q_mean_length.append(np.mean(Q_wave_lengths))
        Q_mean_amplitude.append(np.mean(Q_amplitudes))
        R_wave_lengths = [wave_length(signal=baseline_zero_centered,point=rpeaks["ECG_R_Peaks"][i]) for i in range(len(rpeaks["ECG_R_Peaks"]))]
        R_amplitudes = [baseline_zero_centered[rpeaks["ECG_R_Peaks"][i]] if np.isnan(rpeaks["ECG_R_Peaks"][i])!=True else 0 for i in range(len(rpeaks["ECG_R_Peaks"]))]
        R_mean_length.append(np.mean(R_wave_lengths))
        R_mean_amplitude.append(np.mean(R_amplitudes))
        S_wave_lengths = [wave_length(signal=baseline_zero_centered,point=waves_peak["ECG_S_Peaks"][i]) for i in range(len(waves_peak["ECG_S_Peaks"]))]
        S_amplitudes = [baseline_zero_centered[waves_peak["ECG_S_Peaks"][i]] if np.isnan(waves_peak["ECG_S_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_S_Peaks"]))]
        S_mean_length.append(np.mean(S_wave_lengths))
        S_mean_amplitude.append(np.mean(S_amplitudes))
        T_wave_lengths = [wave_length(signal=baseline_zero_centered,point=waves_peak["ECG_T_Peaks"][i]) for i in range(len(waves_peak["ECG_T_Peaks"]))]
        T_amplitudes = [baseline_zero_centered[waves_peak["ECG_T_Peaks"][i]] if np.isnan(waves_peak["ECG_T_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_T_Peaks"]))]
        T_mean_length.append(np.mean(T_wave_lengths))
        T_mean_amplitude.append(np.mean(T_amplitudes))
        #Extraction of wave_powers
        P_powers = [wave_powers(signal=baseline_zero_centered,point=waves_peak["ECG_P_Peaks"][i]) if np.isnan(waves_peak["ECG_P_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_P_Peaks"]))]
        P_mean_power = np.mean(P_powers)
        P_power.append(P_mean_power)
        P_std_power = np.std(P_powers)
        P_std_powers.append(P_std_power)
        Q_powers= [wave_powers(signal=baseline_zero_centered,point=waves_peak["ECG_Q_Peaks"][i]) if np.isnan(waves_peak["ECG_Q_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_Q_Peaks"]))]
        Q_mean_power = np.mean(Q_powers)
        Q_power.append(Q_mean_power)
        Q_std_power = np.std(Q_powers)
        Q_std_powers.append(Q_std_power)
        R_powers= [wave_powers(signal=baseline_zero_centered,point=rpeaks["ECG_R_Peaks"][i]) if np.isnan(rpeaks["ECG_R_Peaks"][i])!=True else 0 for i in range(len(rpeaks["ECG_R_Peaks"]))]
        R_mean_power = np.mean(R_powers)
        R_power.append(R_mean_power)
        R_std_power = np.std(R_powers)
        R_std_powers.append(R_std_power)
        S_powers= [wave_powers(signal=baseline_zero_centered,point=waves_peak["ECG_S_Peaks"][i]) if np.isnan(waves_peak["ECG_S_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_S_Peaks"]))]
        S_mean_power = np.mean(S_powers)
        S_power.append(S_mean_power)
        S_std_power = np.std(S_powers)
        S_std_powers.append(S_std_power)
        T_powers= [wave_powers(signal=baseline_zero_centered,point=waves_peak["ECG_T_Peaks"][i]) if np.isnan(waves_peak["ECG_T_Peaks"][i])!=True else 0 for i in range(len(waves_peak["ECG_T_Peaks"]))]
        T_mean_power = np.mean(T_powers)
        T_power.append(T_mean_power)
        T_std_power = np.std(T_powers)
        T_std_powers.append(T_std_power)
        #Extraction of distance between peaks
        PQ_dist = np.array([waves_peak['ECG_Q_Peaks'][i]-waves_peak['ECG_P_Peaks'][i] for i in range(len(waves_peak["ECG_P_Peaks"]))])/fs
        mean_PQ_dist = np.mean(PQ_dist)
        std_PQ_dist = np.std(PQ_dist)
        QR_dist = np.array([rpeaks['ECG_R_Peaks'][i]-waves_peak['ECG_Q_Peaks'][i] for i in range(len(rpeaks["ECG_R_Peaks"]))])/fs
        mean_QR_dist = np.mean(QR_dist)
        std_QR_dist = np.std(QR_dist)
        RS_dist = np.array([waves_peak['ECG_S_Peaks'][i]-rpeaks['ECG_R_Peaks'][i] for i in range(len(waves_peak["ECG_S_Peaks"]))])/fs
        mean_RS_dist = np.mean(RS_dist)
        std_RS_dist = np.std(RS_dist)
        ST_dist = np.array([waves_peak['ECG_T_Peaks'][i]-waves_peak['ECG_S_Peaks'][i] for i in range(len(waves_peak["ECG_T_Peaks"]))])/fs
        mean_ST_dist = np.mean(ST_dist)
        std_ST_dist = np.std(ST_dist)
        RT_dist = RS_dist = np.array([waves_peak['ECG_T_Peaks'][i]-rpeaks['ECG_R_Peaks'][i] for i in range(len(waves_peak["ECG_T_Peaks"]))])/fs
        mean_RS_dist = np.mean(RT_dist)
        std_RS_dist = np.std(RT_dist)
    print(P_mean_amplitude)
    df['P_mean_amplitude'] = P_mean_amplitude
    df['Q_mean_amplitude'] = Q_mean_amplitude
    df['R_mean_amplitude'] = R_mean_amplitude
    df['S_mean_amplitude'] = S_mean_amplitude
    df['T_mean_amplitude'] = T_mean_amplitude
    df["P_power"] = P_power
    df["Q_power"] = Q_power
    df["R_power"] = R_power
    df["S_power"] = S_power
    df["T_power"] = T_power
    df["P_std_power"] = P_std_powers
    df["Q_std_power"] = Q_std_powers
    df["R_std_power"] = R_std_powers
    df["S_std_power"] = S_std_powers
    df["T_std_power"] = T_std_powers
    


    return df



In [6]:
# The heartbeats can be found in the ptbxl_database.csv file
heartbeats = df["r_peaks"]
print(len(df))
print(heartbeats)

a = df.iloc[0,:]
print(a["filename_lr"])

current_ecg_peaks = a["r_peaks"] ## SUBSTITUTE BY N or some iteration var
print(type(current_ecg_peaks))
current_ecg_peaks = current_ecg_peaks[1:-1]
curr_ecg_peaks=[int(s) for s in current_ecg_peaks.split()]
x = np.array(curr_ecg_peaks)
x =x*5
print(type(x))
print(x)

21801
0            [ 22 115 208 300 393 488 583 679 773 865 961]
1                            [101 231 368 502 627 747 863]
2            [  7 104 202 298 394 485 577 670 764 857 951]
3        [ 30 108 192 279 365 447 523 601 681 763 841 9...
4            [ 33 123 219 313 399 482 571 669 764 854 938]
                               ...                        
21796    [ 22  79 135 190 245 301 357 413 468 523 579 6...
21797        [ 16 104 192 282 371 460 549 639 731 821 910]
21798                [ 76 175 274 383 491 591 688 791 894]
21799    [ 23 110 199 286 373 458 540 599 703 792 881 967]
21800            [ 68 169 269 369 470 572 674 776 877 980]
Name: r_peaks, Length: 21801, dtype: object
records100/00000/00001_lr
<class 'str'>
<class 'numpy.ndarray'>
[ 110  575 1040 1500 1965 2440 2915 3395 3865 4325 4805]


In [50]:
path = '/Users/alexenabernad/Desktop/Idoven test/Idoven-Data-Scientist/physionet.org/files/ptb-xl/1.0.2/ptbxl_database.csv'
df = pd.read_csv(path)
path_stats = '/Users/alexenabernad/Desktop/Idoven test/Idoven-Data-Scientist/physionet.org/files/ptb-xl/1.0.2/scp_statements.csv'
df_statements = pd.read_csv(path_stats) 
df_statements.rename(columns={'Unnamed: 0': 'Abreviated'}, inplace=True)
# for n in range(len(df)):
#     data = wfdb.rdrecord('/Users/alexenabernad/Desktop/Idoven test/Idoven-Data-Scientist/physionet.org/files/ptb-xl/1.0.2/'+ df["filename_hr"][n])
#     # wfdb.plot_wfdb(record=data, title='Record a103l from PhysioNet Challenge 2015') 
#     display(data.__dict__)
iscal_contained = []
for i in range(len(df)):
    codes = df["scp_codes"][i]
    sr = "ISCAL" in codes   
    iscal_contained.append(sr)
df["ISCAL"] = iscal_contained
normal_contained = []
for i in range(len(df)):
    codes = df["scp_codes"][i]
    sr = "NORM" in codes   
    normal_contained.append(sr)
df["NORMAL"] = normal_contained
df['ISCAL'] = df['ISCAL'].replace([True], 1)
df['ISCAL'] = df['ISCAL'].replace([False], 0)
df['NORMAL'] = df['NORMAL'].replace([True], 1)
df['NORMAL'] = df['NORMAL'].replace([False], 0)
print(len(df))
df_iscal = df[df["ISCAL"]==1]
df_normal = df[df["NORMAL"]==1]
print(len(df_iscal))
#The length of the normal dataframe is much bigger than the iscal cases, for that reason it has to be reduced for a more approximate 1:1 proporiton
df_normal = df_normal.sample(n=700,random_state=6)
df_both = pd.concat([df_iscal,df_normal],axis=0)

df_both = df_both.drop(["height","nurse","site","device","recording_date","report","scp_codes","heart_axis","infarction_stadium1","infarction_stadium2","validated_by","baseline_drift","filename_lr"],axis = 1) #Height is an empty column and can be deleted, nurse column is not relevant, site and device are not relevant instead
# df_both = df_both.drop(["second_opinion"],axis=1)

print(len(df_both))
df_both =df_both.dropna(axis="columns")
df_both =df_both.dropna() # Empty cells are discarded
df_both.reset_index(drop=True,inplace=True) #Reset index of new dataframe with the normal and Iscal cases
print(len(df_both))

print(df_both["filename_hr"])

df_final = extract_features(df=df_both)
print(df_final.head())
df_final.drop(["filename_lr","filename_hr","r_peaks"],axis = 1,inplace=True)
best_var = SelectKBest(score_func=chi2, k = 10)
#Normalize data
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
df_normalized = pd.DataFrame(scaler.fit_transform(df_final),columns = df_final.columns)
y = df_final.iloc[:,-17]
fit = best_var.fit(df_normalized,y)
dfvals = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(df_normalized.columns)
feature_scores = pd.concat([dfcolumns,dfvals],axis = 1)
print(feature_scores)
df_corr = df_final.corr()
sns.heatmap(df_corr)


21801
659
1359
1359
0       records500/00000/00102_hr
1       records500/00000/00135_hr
2       records500/00000/00164_hr
3       records500/00000/00167_hr
4       records500/00000/00168_hr
                  ...            
1354    records500/18000/18286_hr
1355    records500/12000/12900_hr
1356    records500/02000/02827_hr
1357    records500/05000/05803_hr
1358    records500/12000/12034_hr
Name: filename_hr, Length: 1359, dtype: object


{'record_name': '00102_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[ 0.375, -0.1  , -0.475, ..., -1.37 ,  0.305,  0.34 ],
        [ 0.375, -0.1  , -0.475, ..., -1.37 ,  0.305,  0.34 ],
        [ 0.376, -0.1  , -0.476, ..., -1.371,  0.305,  0.34 ],
        ...,
        [ 0.04 ,  0.07 ,  0.03 , ...,  0.085, -0.18 , -0.015],
        [ 0.04 ,  0.07 ,  0.03 , ...,  0.085, -0.18 , -0.015],
        [ 0.04 ,  0.07 ,  0.03 , ...,  0.085, -0.18 , -0.015]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat',
  '00102_hr.dat'],
 'fmt': ['16',
  

{'record_name': '00135_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[ 0.31 ,  0.715,  0.405, ...,  0.485,  1.37 ,  2.26 ],
        [ 0.31 ,  0.715,  0.405, ...,  0.484,  1.369,  2.26 ],
        [ 0.311,  0.715,  0.403, ...,  0.49 ,  1.374,  2.262],
        ...,
        [ 0.   , -0.02 , -0.02 , ...,  0.115,  0.27 ,  0.195],
        [ 0.   , -0.02 , -0.02 , ...,  0.115,  0.27 ,  0.195],
        [ 0.   , -0.02 , -0.02 , ...,  0.115,  0.27 ,  0.195]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat',
  '00135_hr.dat'],
 'fmt': ['16',
  

{'record_name': '00164_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[ 0.08 ,  0.045, -0.035, ..., -0.01 ,  0.2  , -0.09 ],
        [ 0.08 ,  0.045, -0.035, ..., -0.01 ,  0.2  , -0.09 ],
        [ 0.08 ,  0.045, -0.035, ..., -0.01 ,  0.2  , -0.09 ],
        ...,
        [-0.17 , -0.105,  0.065, ..., -0.09 , -0.455, -0.11 ],
        [-0.17 , -0.105,  0.065, ..., -0.09 , -0.455, -0.11 ],
        [-0.17 , -0.105,  0.065, ..., -0.09 , -0.455, -0.11 ]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat',
  '00164_hr.dat'],
 'fmt': ['16',
  

{'record_name': '00167_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[-0.045, -0.09 , -0.045, ...,  0.12 , -0.045, -0.025],
        [-0.045, -0.09 , -0.045, ...,  0.12 , -0.045, -0.025],
        [-0.045, -0.09 , -0.045, ...,  0.12 , -0.045, -0.025],
        ...,
        [ 0.01 , -0.085, -0.095, ..., -0.11 , -0.04 , -0.015],
        [ 0.01 , -0.085, -0.095, ..., -0.11 , -0.04 , -0.015],
        [ 0.01 , -0.085, -0.095, ..., -0.11 , -0.04 , -0.015]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat',
  '00167_hr.dat'],
 'fmt': ['16',
  

...

{'record_name': '03965_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[ 0.005, -0.09 , -0.095, ..., -0.04 ,  0.04 , -0.115],
        [ 0.005, -0.09 , -0.095, ..., -0.04 ,  0.04 , -0.115],
        [ 0.005, -0.09 , -0.095, ..., -0.04 ,  0.04 , -0.115],
        ...,
        [-0.075,  0.035,  0.11 , ..., -0.19 , -0.01 , -0.13 ],
        [-0.075,  0.035,  0.11 , ..., -0.19 , -0.01 , -0.13 ],
        [-0.075,  0.035,  0.11 , ..., -0.19 , -0.01 , -0.13 ]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat',
  '03965_hr.dat'],
 'fmt': ['16',
  

{'record_name': '14259_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[-0.03 , -0.1  , -0.07 , ..., -0.105, -0.17 , -0.065],
        [-0.03 , -0.1  , -0.07 , ..., -0.105, -0.17 , -0.065],
        [-0.03 , -0.1  , -0.07 , ..., -0.105, -0.17 , -0.065],
        ...,
        [-0.01 , -0.09 , -0.08 , ..., -0.065, -0.065,  0.01 ],
        [-0.01 , -0.09 , -0.08 , ..., -0.065, -0.065,  0.01 ],
        [-0.01 , -0.09 , -0.08 , ..., -0.065, -0.065,  0.01 ]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat',
  '14259_hr.dat'],
 'fmt': ['16',
  

{'record_name': '09700_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[ 1.395,  0.93 , -0.465, ...,  0.675,  0.84 ,  0.72 ],
        [ 1.395,  0.93 , -0.465, ...,  0.675,  0.84 ,  0.72 ],
        [ 1.395,  0.931, -0.464, ...,  0.675,  0.84 ,  0.72 ],
        ...,
        [-0.19 , -0.11 ,  0.08 , ...,  0.025, -0.1  , -0.095],
        [-0.19 , -0.11 ,  0.08 , ...,  0.025, -0.1  , -0.095],
        [-0.19 , -0.11 ,  0.08 , ...,  0.025, -0.1  , -0.095]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat',
  '09700_hr.dat'],
 'fmt': ['16',
  

{'record_name': '00888_hr',
 'n_sig': 12,
 'fs': 500,
 'counter_freq': None,
 'base_counter': None,
 'sig_len': 5000,
 'base_time': None,
 'base_date': None,
 'comments': [],
 'sig_name': ['I',
  'II',
  'III',
  'AVR',
  'AVL',
  'AVF',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6'],
 'p_signal': array([[ 0.09 ,  0.115,  0.025, ..., -0.005, -0.035,  0.1  ],
        [ 0.09 ,  0.115,  0.025, ..., -0.005, -0.035,  0.1  ],
        [ 0.09 ,  0.115,  0.025, ..., -0.005, -0.035,  0.1  ],
        ...,
        [-0.315, -0.29 ,  0.025, ..., -0.035, -0.115,  0.   ],
        [-0.315, -0.29 ,  0.025, ..., -0.035, -0.115,  0.   ],
        [-0.315, -0.29 ,  0.025, ..., -0.035, -0.115,  0.   ]]),
 'd_signal': None,
 'e_p_signal': None,
 'e_d_signal': None,
 'file_name': ['00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat',
  '00888_hr.dat'],
 'fmt': ['16',
  

KeyboardInterrupt: 